Ben Trevett 의 [Transformers for Sentiment Analysis](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb) 튜토리얼을 한글 데이터셋에 적용해보는 연습이다. 데이터셋은 네이버 영화 평점 데이터을 이용한다.

이 튜토리얼에서는 BERT 모델([논문](https://arxiv.org/abs/1810.04805))을 이용해서 연습해본다. 이를 위해 Huggingface의 [트랜스포머 라이브러리](https://github.com/huggingface/transformers)를 이용하여 훈련된 단어 벡터를 얻는다. 그후 bidirectional GRU 모델을 이용하여 최종 출력을 얻는다. 

# 전처리

랜덤시드를 고정한다.

In [1]:
import torch
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

트랜스포머의 사전 훈련 벡터를 이용하자. BERT 모델 중 다중언어를 지원하는 'bert-base-multilingual-cased' 토크나이저를 이용한다.

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages

`tokenizer.vocab`에는 사전 훈련된 단어들이 들어있다. 갯수를 확인해보자.

In [3]:
len(tokenizer.vocab)

119547

토큰화 하기 위해서는 `tokenizer.tokenize` 메서드를 이용하면 된다.

In [4]:
tokens = tokenizer.tokenize('우리 사이엔 낮은 담이 있어 서로의 진심을 안을 수가 없어요')
print(tokens)

['우', '##리', '사', '##이', '##엔', '낮', '##은', '담', '##이', '있어', '서로', '##의', '진', '##심을', '안', '##을', '수', '##가', '없', '##어', '##요']


이런 토큰을 인덱스로 변경하려면 `tokenizer.convert_tokens_to_ids` 메서드를 이용한다.

In [5]:
indexes = tokenizer.convert_tokens_to_ids(tokens)
print(indexes)

[9604, 12692, 9405, 10739, 86933, 8992, 10892, 9064, 10739, 45893, 67324, 10459, 9708, 86904, 9521, 10622, 9460, 11287, 9555, 12965, 48549]


트랜스포머 모델은 문장의 시작과 끝을 의미하는 특별한 토큰들 또한 훈련시켜놓았다. 물론 패딩과 미지의 토큰에 대해서도 훈련되어 있다.

In [6]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


이런 특별한 토큰들의 인덱스를 ARABOZA

In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


사실은 좀더 직접적으로 확인할 수도 있습니다 짜잔~

In [8]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


트랜스포머 모델은 문장의 최대 길이가 지정되어 있다. 이는 `max_model_input_sizes`를 통해 확인할 수 있다.

In [9]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-multilingual-cased']
print(max_input_length)

512


기존에는 은전한닢 토크나이저를 이용하였으나 여기서는 직접 함수를 만들어서 최대 길이로 자르고 여기에 추가로 문장 시작 토큰과 마지막 토큰을 고려하여 두개를 추가로 뺀다.

In [10]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

이제 필드를 정의한다. 트랜스포머는 배치 차원을 맨 앞에 놓아야 하므로 `batch_first = True` 옵션을 넣어야 한다. 그리고 단어장을 이미 만들었으므로 `use_vocab = False`를 선언한다. 토크나이저로는 위에서 정의한 `tokenize_and_cut` 함수를 이용한다. `preprocessing` 인수는 토크나이징 된 이후 적용할 함수를 입력하는 곳이며, 여기서는 토큰을 인덱스로 변환하는 메서드를 넣는다. 마지막으로 특수 토큰들을 인덱스로 선언한다(이미 후처리에서 인덱스로 변환되었으므로).

In [11]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                 use_vocab = False,
                 tokenize = tokenize_and_cut,
                 preprocessing = tokenizer.convert_tokens_to_ids,
                 init_token = init_token_idx,
                 eos_token = eos_token_idx,
                 pad_token = pad_token_idx,
                 unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

데이터를 불러온 후 분리한다.

In [12]:
fields = {'text': ('text',TEXT), 'label': ('label',LABEL)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [13]:
train_data, test_data = data.TabularDataset.splits(
                            path = 'data',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [14]:
print(f'훈련 데이터 갯수: {len(train_data)}')
print(f'검증 데이터 갯수: {len(valid_data)}')
print(f'테스트 데이터 갯수: {len(test_data)}')

훈련 데이터 갯수: 104996
검증 데이터 갯수: 44999
테스트 데이터 갯수: 49997


실제 데이터가 인덱스로 잘 변환되었는지 확인해보자.

In [15]:
print(vars(train_data.examples[10]))

{'text': [9574, 119439, 10459, 18622, 42815, 12638, 30842, 14423, 48418, 11664, 80331, 10622, 9056, 35866, 11018, 119259, 12178, 117, 100, 119, 9428, 12030, 85903, 16439, 9729, 26737, 17342], 'label': '0'}


실제 어떤 문장이었는지 확인해보자.

In [16]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[10])['text'])
print(tokens)

['영', '##혼', '##의', '##치', '##유', '##와', '##현', '##대', '##인의', '##고', '##독', '##을', '다', '##루', '##는', '##척', '##하는', ',', '[UNK]', '.', '선', '##인', '##장에', '##나', '찔', '##려', '##라']


In [17]:
string = tokenizer.convert_tokens_to_string(tokens)
print(string)

영혼의치유와현대인의고독을 다루는척하는 , [UNK] . 선인장에나 찔려라


텍스트의 단어장은 만들었으나 라벨 단어장은 만들어야 한다.

In [18]:
LABEL.build_vocab(train_data)

In [19]:
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


이터레이터를 만들자. 배치 사이즈는 가능한 크게 하는게 트랜스포머의 성능을 올리는 데 도움이 되는 편이다.

In [37]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

# 모델 생성

토크나이저와 동일하게 모델도 사전 훈련된 모델을 불러오자.

In [21]:
from transformers import BertModel

bert = BertModel.from_pretrained('bert-base-multilingual-cased')

이제 진짜 모델을 정의하자. 임베딩 레이어는 앞서 정의한 트랜스포머 모델로 대체한다. 이후 임베딩을 GRU 에 넣어서 결과를 출력하도록 한다. 이때 임베딩 차원(`hidden_size`)은 트랜스포머의 `config`에서 가져온다.

순전파 과정에서 트랜스포머를 `no_grad`로 감싸서 훈련되지 않도록 한다. 트랜스포머 모델은 원래 pooling된 결과값을 반환하게 되어 있는데, 여기서는 쓰지 않겠다. 이 외에는 일반적인 RNN 모델의 구조이다.

In [22]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim,
                n_layers, bidirectional, dropout):
        super().__init__()
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.rnn = nn.GRU(embedding_dim, hidden_dim,
                         num_layers = n_layers,
                         bidirectional = bidirectional,
                         batch_first = True,
                         dropout = 0 if n_layers <2 else dropout)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional
                            else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch_size, sent_len]
        with torch.no_grad():
            embedded = self.bert(text)[0]    
        #embedded = [batch_size, sent_len, emb_dim]

        _, hidden = self.rnn(embedded)
        #hideen = [n_layers * n_directions, batch_size, emb_dim]
        
        if self.rnn.bidirectional:
            # 마지막 레이어의 양방향 히든 벡터만 가져옴
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        #hidden = [batch_size, hid_dim]
        
        output = self.out(hidden)
        #output = [batch_size, out_dim]
        
        return output

하이퍼파라미터를 지정하고 모델을 생성하자.

In [23]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert, HIDDEN_DIM, OUTPUT_DIM,
                        N_LAYERS, BIDIRECTIONAL, DROPOUT)

모델의 파라미터 수를 확인해보자. 

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'모델의 파라미터 수는 {count_parameters(model):,}, 이 중 버트 모델의 파라미터 수는 {count_parameters(bert):,}개입니다.')

모델의 파라미터 수는 180,612,609, 이 중 버트 모델의 파라미터 수는 177,853,440개입니다.


버트 모델의 파라미터는 훈련시키지 않기 위해 얼리자.

In [26]:
for name, param in model.named_parameters():

    if name.startswith('bert'):
        param.requires_grad = False

이제 실제 훈련시켜야 하는 모델의 파라미터 수를 확인해보자.

In [27]:
print(f'모델의 파라미터 수는 {count_parameters(model):,}개입니다.')

모델의 파라미터 수는 2,759,169개입니다.


따라서 기존 모델들과 큰 차이 없는 수준의 파라미터만 훈련시키면 되는 것을 확인할 수 있다. 그러나 버트 모델을 통과하는 시간이 있으므로 훨씬 오래 걸리기는 한다.

혹시 모르니 모델 구조를 확인해보자.

In [29]:
for name, param in model.named_parameters():

    if param.requires_grad == True:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


# 모델 훈련

기존과 유사하게 하자.

In [30]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [31]:
model = model.to(device)
criterion = criterion.to(device)

기존에 하던 대로...

In [32]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

훈련 함수를 정의하자. 여기선 드랍아웃 안쓰지만 걍 `model.train()` 사용하겠다.

In [33]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1) # output_dim = 1
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [34]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

얼마나 훈련 걸리는 지 체크하는 함수

In [35]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

이제 훈련시켜보자.


In [38]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 4m 14s
	Train Loss: 0.441 | Train Acc: 78.95%
	 Val. Loss: 0.420 |  Val. Acc: 80.73%
Epoch: 02 | Epoch Time: 4m 15s
	Train Loss: 0.395 | Train Acc: 81.64%
	 Val. Loss: 0.380 |  Val. Acc: 82.69%
Epoch: 03 | Epoch Time: 4m 15s
	Train Loss: 0.370 | Train Acc: 83.16%
	 Val. Loss: 0.367 |  Val. Acc: 83.47%
Epoch: 04 | Epoch Time: 4m 15s
	Train Loss: 0.350 | Train Acc: 84.13%
	 Val. Loss: 0.371 |  Val. Acc: 83.59%
Epoch: 05 | Epoch Time: 4m 15s
	Train Loss: 0.332 | Train Acc: 85.16%
	 Val. Loss: 0.371 |  Val. Acc: 84.18%


역시 도란스포머다...

테스트셋에서 돌려보자.

In [39]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.368 | Test Acc: 83.37%


추가훈련!

In [45]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+6:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 06 | Epoch Time: 4m 14s
	Train Loss: 0.348 | Train Acc: 84.23%
	 Val. Loss: 0.371 |  Val. Acc: 83.82%
Epoch: 07 | Epoch Time: 4m 15s
	Train Loss: 0.332 | Train Acc: 85.05%
	 Val. Loss: 0.358 |  Val. Acc: 84.01%
Epoch: 08 | Epoch Time: 4m 15s
	Train Loss: 0.317 | Train Acc: 85.86%
	 Val. Loss: 0.353 |  Val. Acc: 84.29%
Epoch: 09 | Epoch Time: 4m 15s
	Train Loss: 0.306 | Train Acc: 86.46%
	 Val. Loss: 0.357 |  Val. Acc: 84.38%
Epoch: 10 | Epoch Time: 4m 15s
	Train Loss: 0.295 | Train Acc: 86.91%
	 Val. Loss: 0.356 |  Val. Acc: 84.78%


# 추론 



In [40]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [41]:
predict_sentiment(model, tokenizer, "이 영화 진짜 재밌었다!!")

0.9618139863014221

In [42]:
predict_sentiment(model, tokenizer, "영화관에서 이걸 본 내가 바보다. 내 돈 돌려줘!!!")

0.05189424753189087

In [43]:
predict_sentiment(model, tokenizer, "이 영화 감독 밥은 먹고 다니냐? 이런 영화 만들고 잠이 와?")

0.03582892194390297

In [44]:
predict_sentiment(model, tokenizer, "내 인생 영화 등극. 주인공한테 너무 몰입해서 시간 가는 줄도 몰랐다...")

0.7696210145950317

굿굿!